# You Work at the North Pole
Every day, little children around the world write letters to Santa telling him what they want for christmas. Santa reads these letters, and if the child is nice, the gift gets made by one of the elves, wrapped, and put in santa's sleigh to be delivered on Christmas Eve. If the child is naughty, Santa puts coal in their stocking. Coal doesn't need to be wrapped, but it does need to be put in Santa's sleigh.

In an attempt to teach myself threading in python, I'm going to create a working, multi-threaded model of Santa's workshop. 

## All By My Elf
For now, let's assume Santa's workshop has undergone some budget cuts, and only has one elf employed. Making a present takes a variable amount of time--let's say between 0 (maybe the child wished for no presents?) and anything less than 1 hour. Wrapping a present takes a constant amount of time: 10 minutes. For now, we're just going to assume all children are good, so these two tasks can be represented by the following functions:

In [1]:
import time
import random

def make_wish_come_true(wish):
    time.sleep(random.random())
    return wish
        
def wrap_present(present):
    time.sleep(0.1)
    return present

So to fill Santa's sleigh, our lone elf gathers the letters, makes the toys, wraps the toys, and puts them in the sleigh. If a child is naughty, we'll just toss some coal directly into the sleigh; no use wasting an elf's precious time on a bratty kid.

In [2]:
import queue

def fill_santas_sleigh(childrens_letters, santas_list):
    # Santa gives nice kids' letters to elves so they can make them into toys.
    letters = []
    # Finished toys sit in a pile waiting to be wrapped.
    toys = []
    # Finished, wrapped toys.
    sleigh = []
    
    for child, wish in childrens_letters.items():
        # He's makin' a list...checkin' it twice!
        if santas_list[child] == "Naughty" and santas_list[child] == "Naughty":
            sleigh.append("Coal for " + child)
        else:
            letters.append(wish + " for " + child)
            
    while letters:
        toy = make_wish_come_true(letters.pop())
        toys.append(toy)

    while toys:
        present = wrap_present(toys.pop())
        sleigh.append(present)
    
    return sleigh

To make sure this all works the way we expect it to, here's some sample data and a test to run it through!

In [3]:
childrens_letters_2017 = {
    "Jimmy":"Racecar",
    "Alison":"Nerf gun",
    "Steve":"Bicycle",
    "Gwendylyn":"Skateboard",
    "Lewis":"Action figure",
    "Kate":"Nintendo",
    "Alex":"Coloring book",
    "Sue":"Stuffed animal"
}

santas_list_2017 = {
    "Jimmy":"Nice",
    "Alison":"Nice",
    "Steve":"Naughty",
    "Gwendylyn":"Nice",
    "Lewis":"Naughty",
    "Kate":"Naughty",
    "Alex":"Nice",
    "Sue":"Nice"
}

sleigh_2017 = fill_santas_sleigh(childrens_letters_2017, santas_list_2017)

print (sleigh_2017)

['Coal for Steve', 'Coal for Lewis', 'Coal for Kate', 'Racecar for Jimmy', 'Nerf gun for Alison', 'Skateboard for Gwendylyn', 'Coloring book for Alex', 'Stuffed animal for Sue']


## Thread The Halls
Ok, so now the workshop is back in the black, and we now have a bunch of elves, not just one. But our `fill_santas_sleigh` function still operates the same, and the new recruits are getting bored. Wouldn't it be great if multiple elves could make and wrap presents in parallel? 

This is where threading comes in. Each elf is a thread, and it can either make or wrap a gift. So if we have "n" elves, we would want each elf to operate like this:

1. If there are toys lying around waiting to be wrapped, wrap them and put them in the sleigh.
2. If there aren't any toys made, go get a letter from santa and make that wish come true.
3. If everything's made, wrapped, and in the sleigh, *exit* the workshop.

There's an important caveat to realize here: if there are multiple elves looking for something to do, they need to do different things. For example, we don't want our little workers bickering over letters or breaking toys because they're both trying to wrap the same one. The solution? Locks. 

We want to make sure only one elf talks to Santa at a time, so if Santa is already busy giving a letter to another elf, other elves should quietly wait their turn. We'll call this a `letter_lock`. Similarly, to avoid fights with elves wanting to wrap the same toy (or elves getting hit by toys being thrown onto the pile), only one elf should approach the pile of toys at a time. The other elves *politely waiting their turn*--meaning we'll also have a `toy_lock`. Finally, we don't want elves throwing toys into the sleigh at the same time--they could collide and break! Christmas would be ruined! To avoid this catastrophe, we'll have a `sleigh_lock`.

> ### A quick note on locks...
> Locks are really important if the resource you're trying to access isn't threadsafe. Lists are a good example of this, but it's worth noting that there are other classes (queues come to mind) that are indeed threadsafe, so you don't have to bother with locks. Therefore, I'm intentionally choosing a sub-optimal data structure to more completely illustrate how multithreading works.

\*cough cough\* With that non-metaphor-fitting blabber out of the way...let's create our own Elf class that is a subclass of Thread (found in the threading python module). During each elf's orientation training (`__init__` for all you programmers), he'll be given a badge with his name, get shown where to find Santa (who has the children's letters), the pile of toys, and Santa's sleigh. He'll also be told about the one-elf-at-a-time policies for each.

In [4]:
import threading

class Elf (threading.Thread):
    def __init__(self, name, letter_location, toy_location, sleigh_location, letter_policy, toy_policy, sleigh_policy):
        threading.Thread.__init__(self)
        self.name = name
        self.letters = letter_location
        self.toys = toy_location
        self.sleigh = sleigh_location
        self.turn_for_letter = letter_policy
        self.turn_for_toy = toy_policy
        self.turn_for_sleigh = sleigh_policy
        print (self.name, "is ready to make some toys!")
    
    def run(self):
        while self.toys or self.letters:
            # Grab a toy and let the next elf take their turn.
            toy = None
            with self.turn_for_toy:
                if self.toys:
                    toy = self.toys.pop()
            # Wrap that toy and put it in the sleigh.
            if toy:
                present = wrap_present(toy)
                with self.turn_for_sleigh:
                    self.sleigh.append(present)
#                 print (self.name, "wrapped a", present, "and threw it in the sleigh")
                time.sleep(random.random())
                continue

            # Grab a letter and let the next elf take their turn.
            wish = None
            with self.turn_for_letter:
                if self.letters:
                    wish = self.letters.pop()
            # Make a toy and put it in the pile.
            if wish:
                toy = make_wish_come_true(wish)
                with self.turn_for_toy:
                    self.toys.append(toy)
#                 print (self.name, "made a", wish, "and put it in the toy pile")
                time.sleep(random.random())
                continue
                
        print (self.name, "is punching out for the day!")

You'll notice in the code above that our elf class has a `run` method that follows the 3 steps we outlined above for how an elf should behave. When the function finishes, the elf (thread) exits. Notice this also holds a lot of the code we'd previously placed in `fill_santas_sleigh`, which means we'll want to modify that code as well. The multi-threaded code looks like this:

In [5]:
santa_lock = threading.Lock()
toy_lock = threading.Lock()
sleigh_lock = threading.Lock()

def everyone_fill_santas_sleigh(childrens_letters, santas_list, elves_names):
    # Santa gives nice kids' letters to elves so they can make them into toys.
    letters = []
    # Finished toys sit in a pile waiting to be wrapped.
    toys = []
    # Finished, wrapped toys.
    sleigh = []
    
    for child, wish in childrens_letters.items():
        # He's makin' a list...checkin' it twice!
        if santas_list[child] == "Naughty" and santas_list[child] == "Naughty":
            sleigh.append("coal for " + child)
        else:
            letters.append(wish + " for " + child)
    
    elves = []
    for name in elves_names:
        elf = Elf(name, letters, toys, sleigh, santa_lock, toy_lock, sleigh_lock)
        elf.start()
        elves.append(elf)

    # Wait for all the elves to finish working before closing the workshop for the day.
    for elf in elves:
        elf.join()
    
    time.sleep(0.1)
    print ("All clear! Shutting down the workshop for the day.")
    return sleigh

# It's A Christmas Miracle!
That's everything required for a multi-threaded version of the North Pole. To make sure everything's behaving as expected, here's our previous example, this time with the addition of 3 elves.

In [6]:
letters_2017 = {
    "Jimmy":"Racecar",
    "Alison":"Nerf gun",
    "Steve":"Bicycle",
    "Gwendylyn":"Skateboard",
    "Lewis":"Action figure",
    "Kate":"Nintendo",
    "Alex":"Coloring book",
    "Sue":"Stuffed animal"
}

santas_list_2017 = {
    "Jimmy":"Nice",
    "Alison":"Nice",
    "Steve":"Naughty",
    "Gwendylyn":"Nice",
    "Lewis":"Naughty",
    "Kate":"Naughty",
    "Alex":"Nice",
    "Sue":"Nice"
}

elves_names_2017 = {
    "Flint",
    "Peabody",
    "Keebler"
}

sleigh_2017 = everyone_fill_santas_sleigh(letters_2017, santas_list_2017, elves_names_2017)

print (sleigh_2017)

Keebler is ready to make some toys!
Peabody is ready to make some toys!
Flint is ready to make some toys!
Keebler is punching out for the day!
Peabody is punching out for the day!
Flint is punching out for the day!
All clear! Shutting down the workshop for the day.
['coal for Steve', 'coal for Lewis', 'coal for Kate', 'Skateboard for Gwendylyn', 'Coloring book for Alex', 'Stuffed animal for Sue', 'Nerf gun for Alison', 'Racecar for Jimmy']


# ...And To All A Good Night
Just in case you wanna do some stress testing, here's a test that runs through a list of over 4500 nouns! (It's updated regularly, so it keeps growing)

In [7]:
from urllib.request import urlopen

# Some constants to help control the size of the test.
NUMBER_OF_ELVES = 100
MAX_NUMBER_OF_LETTERS = 4000

# This gets a big list of nouns and puts it in a list of wishes.
word_site = "http://www.desiquintans.com/downloads/nounlist/nounlist.txt"
response = urlopen(word_site)
noun_list = response.read().splitlines()
wishes = random.sample(noun_list, min(MAX_NUMBER_OF_LETTERS, len(noun_list)))

santas_elves = list(map(lambda num: "elf" + str(num), range(NUMBER_OF_ELVES)))
childrens_letters = {}
naughty_or_nice_list = {}

# Generate the children's letters and Santa's naughty/nice list.
for i, wish in enumerate(wishes):
    childs_name = "child" + str(i)
    child_is_nice = random.choice([True, True, True, False]) # Most children are nice
    childrens_letters[childs_name] = wish.decode('utf-8') # Decode gets rid of b' prefix
    naughty_or_nice_list[childs_name] = "Nice" if child_is_nice else "Naughty"

big_ass_sleigh = everyone_fill_santas_sleigh(childrens_letters, naughty_or_nice_list, santas_elves)
print (big_ass_sleigh)

elf0 is ready to make some toys!
elf1 is ready to make some toys!
elf2 is ready to make some toys!
elf3 is ready to make some toys!
elf4 is ready to make some toys!
elf5 is ready to make some toys!
elf6 is ready to make some toys!
elf7 is ready to make some toys!
elf8 is ready to make some toys!
elf9 is ready to make some toys!
elf10 is ready to make some toys!
elf11 is ready to make some toys!
elf12 is ready to make some toys!
elf13 is ready to make some toys!
elf14 is ready to make some toys!
elf15 is ready to make some toys!
elf16 is ready to make some toys!
elf17 is ready to make some toys!
elf18 is ready to make some toys!
elf19 is ready to make some toys!
elf20 is ready to make some toys!
elf21 is ready to make some toys!
elf22 is ready to make some toys!
elf23 is ready to make some toys!
elf24 is ready to make some toys!
elf25 is ready to make some toys!
elf26 is ready to make some toys!
elf27 is ready to make some toys!
elf28 is ready to make some toys!
elf29 is ready to make s